# Google Mobility Data
# NYT COVID Cases & Death Data


In [1]:
# Dependencies and Setup
import json
import os
import pandas as pd
import urllib.request
import requests
from config import db_user, db_pwd
from sqlalchemy import create_engine

## Store Google CSV into DataFrame

In [2]:
csv_file = "Resources/google_mob_US.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df.head()

,Unnamed: 0,State,date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,0,Alabama,2020-02-15,5.0,2.0,39.0,7.0,2.0,-1.0
1,1,Alabama,2020-02-16,0.0,-2.0,-7.0,3.0,-1.0,1.0
2,2,Alabama,2020-02-17,3.0,0.0,17.0,7.0,-17.0,4.0
3,3,Alabama,2020-02-18,-4.0,-3.0,-11.0,-1.0,1.0,2.0
4,4,Alabama,2020-02-19,4.0,1.0,6.0,4.0,1.0,0.0


### Create a new dataframe with select columns

In [3]:
# Create a dataframe with only the columns you need from the original dataframe:
google_df = google_data_df[["State", "date", "retail_and_recreation",
                            "grocery_and_pharmacy", "parks"]].copy()
google_df

,State,date,retail_and_recreation,grocery_and_pharmacy,parks
0,Alabama,2020-02-15,5.0,2.0,39.0
1,Alabama,2020-02-16,0.0,-2.0,-7.0
2,Alabama,2020-02-17,3.0,0.0,17.0
3,Alabama,2020-02-18,-4.0,-3.0,-11.0
4,Alabama,2020-02-19,4.0,1.0,6.0
...,...,...,...,...,...
456457,Wyoming,2020-07-29,NaN,NaN,NaN
456458,Wyoming,2020-07-30,NaN,NaN,NaN
456459,Wyoming,2020-07-31,NaN,NaN,NaN
456460,Wyoming,2020-08-03,NaN,NaN,NaN


In [4]:
google_us = google_df.rename(columns = {"State":'State', "date":'Date', "retail_and_recreation":'Retail_Recreation',
                            "grocery_and_pharmacy":'Grocery_Pharmacy', "parks":'Parks'})
google_us

,State,Date,Retail_Recreation,Grocery_Pharmacy,Parks
0,Alabama,2020-02-15,5.0,2.0,39.0
1,Alabama,2020-02-16,0.0,-2.0,-7.0
2,Alabama,2020-02-17,3.0,0.0,17.0
3,Alabama,2020-02-18,-4.0,-3.0,-11.0
4,Alabama,2020-02-19,4.0,1.0,6.0
...,...,...,...,...,...
456457,Wyoming,2020-07-29,NaN,NaN,NaN
456458,Wyoming,2020-07-30,NaN,NaN,NaN
456459,Wyoming,2020-07-31,NaN,NaN,NaN
456460,Wyoming,2020-08-03,NaN,NaN,NaN


In [5]:
grouped_google_df = pd.DataFrame(google_us.groupby("State").sum())
grouped_google_df.head()

,Retail_Recreation,Grocery_Pharmacy,Parks
State,,,
Alabama,-55224.0,34067.0,51333.0
Alaska,257.0,5424.0,28191.0
Arizona,-43306.0,-4715.0,7218.0
Arkansas,-25510.0,34334.0,56619.0
California,-221017.0,-13869.0,80799.0


In [6]:
google_df = grouped_google_df.reset_index()
google_df.head()

,State,Retail_Recreation,Grocery_Pharmacy,Parks
0,Alabama,-55224.0,34067.0,51333.0
1,Alaska,257.0,5424.0,28191.0
2,Arizona,-43306.0,-4715.0,7218.0
3,Arkansas,-25510.0,34334.0,56619.0
4,California,-221017.0,-13869.0,80799.0


## Store NYT COVID cases and deaths CSV into DataFrame

In [7]:
csv_file = "Resources/COVID-states.csv"
covid_data_df = pd.read_csv(csv_file)
covid_data_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [8]:
covid_us = covid_data_df.rename(columns = {"state":'State', "fips": 'Fips', "date":'Date', "cases":'Cases',
                            "deaths":'Deaths'})
covid_us.head()

,Date,State,Fips,Cases,Deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [9]:
grouped_covid_df = pd.DataFrame(covid_us.groupby("State").sum())
grouped_covid_df.head()

,Fips,Cases,Deaths
State,,,
Alabama,194,9593940,190913
Alaska,390,420666,2930
Arizona,964,16491289,396924
Arkansas,980,4741962,62148
California,1452,55553636,1181987


In [25]:
covid.State=covid.State.astype('str')

In [26]:
covid = grouped_covid_df.reset_index()
covid.head()

,State,Fips,Cases,Deaths
0,Alabama,194,9593940,190913
1,Alaska,390,420666,2930
2,Arizona,964,16491289,396924
3,Arkansas,980,4741962,62148
4,California,1452,55553636,1181987


In [27]:
covid.dtypes

State     object
Fips       int64
Cases      int64
Deaths     int64
dtype: object

### Combine the data into a single dataset

In [12]:
# # Combine the data into a single dataset
us_df = pd.merge(google_us, covid_us, how ='inner', on = "State")

# # Display the data table for preview
us_df

,State,Date_x,Retail_Recreation,Grocery_Pharmacy,Parks,Date_y,Fips,Cases,Deaths
0,Alabama,2020-02-15,5.0,2.0,39.0,2020-03-13,1,6,0
1,Alabama,2020-02-15,5.0,2.0,39.0,2020-03-14,1,12,0
2,Alabama,2020-02-15,5.0,2.0,39.0,2020-03-15,1,23,0
3,Alabama,2020-02-15,5.0,2.0,39.0,2020-03-16,1,29,0
4,Alabama,2020-02-15,5.0,2.0,39.0,2020-03-17,1,39,0
...,...,...,...,...,...,...,...,...,...
94271326,Wyoming,2020-08-04,NaN,NaN,NaN,2020-09-18,56,4747,49
94271327,Wyoming,2020-08-04,NaN,NaN,NaN,2020-09-19,56,4780,49
94271328,Wyoming,2020-08-04,NaN,NaN,NaN,2020-09-20,56,4871,49
94271329,Wyoming,2020-08-04,NaN,NaN,NaN,2020-09-21,56,4944,49


### Connect to local database

In [14]:
# rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
rds_connection_string = f"{db_user}:{db_pwd}@localhost:5432/mobility_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [18]:
engine.table_names()

['google_data', 'covid_data']

### Use pandas to load csv converted DataFrame into database

In [28]:
covid.to_sql(name='covid_data', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "State" of relation "covid_data" does not exist
LINE 1: INSERT INTO covid_data ("State", "Fips", "Cases", "Deaths") ...
                                ^

[SQL: INSERT INTO covid_data ("State", "Fips", "Cases", "Deaths") VALUES (%(State)s, %(Fips)s, %(Cases)s, %(Deaths)s)]
[parameters: ({'State': 'Alabama', 'Fips': 194, 'Cases': 9593940, 'Deaths': 190913}, {'State': 'Alaska', 'Fips': 390, 'Cases': 420666, 'Deaths': 2930}, {'State': 'Arizona', 'Fips': 964, 'Cases': 16491289, 'Deaths': 396924}, {'State': 'Arkansas', 'Fips': 980, 'Cases': 4741962, 'Deaths': 62148}, {'State': 'California', 'Fips': 1452, 'Cases': 55553636, 'Deaths': 1181987}, {'State': 'Colorado', 'Fips': 1616, 'Cases': 6043344, 'Deaths': 252657}, {'State': 'Connecticut', 'Fips': 1791, 'Cases': 7220547, 'Deaths': 628760}, {'State': 'Delaware', 'Fips': 1960, 'Cases': 1935382, 'Deaths': 71818}  ... displaying 10 of 55 total bound parameter sets ...  {'State': 'Wiscons

### Confirm data has been added by querying the table
* NOTE: can also check using pgAdmin

In [ ]:
# pd.read_sql_query('select * from ____', con=engine).head()

### Confirm data has been added by querying the customer_location table

In [ ]:
pd.read_sql_query('select * from _______', con=engine).head()

In [ ]:
# Remove the total column to not interfere with the data represented on the plot
df_low_5_states = df_low_5_states.drop(columns=['total'])

In [ ]:
# Use the tranpose function to transpose the data so that dates become rows
df_low_5_states_t = df_low_5_states.T

In [ ]:
# Plot the mobility data for the top 5 states
df_low_5_states_t.plot.line(figsize=(15, 10))
plt.title('Mobility Trends in the Lowest 5 States',size = 20)
plt.xlabel("Timepoints (days)", size = 25)
plt.ylabel("Change from baseline", size = 25)
plt.hlines(100,0,220,alpha = 0.75)
plt.grid(linestyle ="-", linewidth = 1, alpha = 0.3)
plt.legend(loc = "upper left", fontsize = "large")
plt.savefig('Mobility Trends in the Lowest 5 States.png')

## Filtering the data for the month of May

#### Mobility Data for Top 5 States - Month of May

In [ ]:
top_5 = df_top_5_states_t.reset_index()

In [ ]:
top_5["index"] = (top_5["index"].astype("datetime64")).dt.strftime("%m-%d")
top_5.head()

In [ ]:
# Filtering US Transportation Type data by specified date range
start_date = "05-01"
end_date = "05-30"
mask = (top_5['index'] > start_date) & (top_5['index'] <= end_date)
top_5_may_df = top_5.loc[mask]
top_5_may_df.head()

### Mobility Data for lowest 5 States - Month of May

In [ ]:
low_5 = df_low_5_states_t.reset_index()

In [ ]:
low_5["index"] = (low_5["index"].astype("datetime64")).dt.strftime("%m-%d")
low_5.head()

In [ ]:
# Filtering US Transportation Type data by specified date range
start_date = "05-01"
end_date = "05-30"
mask = (low_5['index'] > start_date) & (low_5['index'] <= end_date)
low_5_may_df = low_5.loc[mask]
low_5_may_df.head()